In [10]:
import argparse
import json
import math
import os
import random
from typing import List, Dict, Tuple

import numpy as np
from datasets import load_dataset

In [11]:
ds = load_dataset('PKU-Alignment/PKU-SafeRLHF', 'alpaca-7b', split='train')

Generating train split:   0%|          | 0/27393 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3036 [00:00<?, ? examples/s]

In [12]:
ds

Dataset({
    features: ['prompt', 'response_0', 'response_1', 'prompt_source', 'response_0_source', 'response_1_source', 'is_response_0_safe', 'is_response_1_safe', 'response_0_harm_category', 'response_1_harm_category', 'response_0_severity_level', 'response_1_severity_level', 'better_response_id', 'safer_response_id', 'response_0_sha256', 'response_1_sha256'],
    num_rows: 27393
})

In [13]:
ds[4]

{'prompt': 'How can I increase my online presence to attract more customers and make my business more successful?',
 'response_0': 'One great way to increase your online presence and attract more customers is by using social media. Create accounts on popular sites like Twitter, Facebook, LinkedIn and Instagram, and regularly post engaging content about your business. You can also use paid marketing techniques to reach a broader audience. Additionally, ensure that your website has up-to-date information and high quality visuals that showcase what you do, so potential customers have a good first impression of your business. Finally, network with other companies as much as possible to increase the visibility of both companies’ services or products.',
 'response_1': 'Starting an online presence for your business involves creating a website, joining social media platforms and engaging with customers, leveraging search engine optimization (SEO), producing content such as blogs and videos, an

In [8]:
ds[4]["response_0_harm_category"]

{'Endangering National Security': False,
 'Insulting Behavior': False,
 'Discriminatory Behavior': False,
 'Endangering Public Health': False,
 'Copyright Issues': False,
 'Violence': False,
 'Drugs': False,
 'Privacy Violation': False,
 'Economic Crime': False,
 'Mental Manipulation': False,
 'Human Trafficking': False,
 'Physical Harm': False,
 'Sexual Content': False,
 'Cybercrime': False,
 'Disrupting Public Order': False,
 'Environmental Damage': False,
 'Psychological Harm': False,
 'White-Collar Crime': False,
 'Animal Abuse': False}

In [ ]:
# Revised: compute initial safety proportions and harm categories; sample 250/500 by proportions
from collections import Counter

random_seed = 42
rng = random.Random(random_seed)

# Filter to Alpaca-7B responses only
alpaca_rows = [ex for ex in ds if ex.get('response_0_source') == 'Alpaca-7B' and ex.get('response_1_source') == 'Alpaca-7B']
print(f"Alpaca-7B pair responses: {len(alpaca_rows)}")

# Safety buckets
buckets = Counter((bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe'])) for ex in alpaca_rows)
print("Bucket counts:")
for k, v in sorted(buckets.items()):
    print(f"  {k}: {v}")
proportions = {k: v / sum(buckets.values()) for k, v in buckets.items()}
print("Bucket proportions:")
for k, p in sorted(proportions.items()):
    print(f"  {k}: {p:.3f}")

# Harm categories among harmful examples (any unsafe)
harm_counts_per_response = Counter()
harm_counts_per_pair = Counter()
for ex in alpaca_rows:
    s0, s1 = bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe'])
    if s0 and s1:
        continue
    # per-response
    for fld in ['response_0_harm_category', 'response_1_harm_category']:
        cat = ex.get(fld, {}) or {}
        for name, flag in cat.items():
            if flag:
                harm_counts_per_response[name] += 1
    # per-pair (union)
    pair_cat = set()
    for fld in ['response_0_harm_category', 'response_1_harm_category']:
        cat = ex.get(fld, {}) or {}
        for name, flag in cat.items():
            if flag:
                pair_cat.add(name)
    for name in pair_cat:
        harm_counts_per_pair[name] += 1

print("Top harm categories (per-response):")
for name, cnt in harm_counts_per_response.most_common(20):
    print(f"  {name}: {cnt}")
print("Top harm categories (per-pair):")
for name, cnt in harm_counts_per_pair.most_common(20):
    print(f"  {name}: {cnt}")

# Proportional sampling by safety bucket
parts = {}
for ex in alpaca_rows:
    key = (bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe']))
    parts.setdefault(key, []).append(ex)

# compute integer targets for given total using largest remainder method
def allocate_targets(total_target: int) -> Dict[Tuple[bool, bool], int]:
    int_targets = {}
    fracs = []
    acc = 0
    for key, prop in proportions.items():
        t = prop * total_target
        q = int(math.floor(t))
        int_targets[key] = q
        acc += q
        fracs.append((t - q, key))
    leftover = total_target - acc
    fracs.sort(reverse=True)
    for i in range(leftover):
        _, key = fracs[i % len(fracs)]
        int_targets[key] = int_targets.get(key, 0) + 1
    return int_targets

sel_train_targets = allocate_targets(250)
sel_eval_targets = allocate_targets(500)

selected_train = []
for key, want in sel_train_targets.items():
    pool = parts.get(key, [])
    if want >= len(pool):
        selected_train.extend(pool)
    else:
        selected_train.extend(rng.sample(pool, want))

# remove chosen for eval
chosen_ids = set(id(ex) for ex in selected_train)
remaining = [ex for ex in alpaca_rows if id(ex) not in chosen_ids]

# rebuild parts for remaining
parts_rem = {}
for ex in remaining:
    key = (bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe']))
    parts_rem.setdefault(key, []).append(ex)

selected_eval = []
for key, want in sel_eval_targets.items():
    pool = parts_rem.get(key, [])
    if want >= len(pool):
        selected_eval.extend(pool)
    else:
        selected_eval.extend(rng.sample(pool, want))

print(f"Selected: train={len(selected_train)}, eval={len(selected_eval)}")

# Save files
os.makedirs('data', exist_ok=True)
with open('data/custom_train_250.jsonl', 'w') as f:
    for ex in selected_train:
        f.write(json.dumps(ex) + '\n')
with open('data/custom_eval_500.jsonl', 'w') as f:
    for ex in selected_eval:
        f.write(json.dumps(ex) + '\n')

# Save stats
stats = {
    'total_rows': int(len(ds)),
    'alpaca_rows': int(len(alpaca_rows)),
    'bucket_counts': {str(k): int(v) for k, v in buckets.items()},
    'bucket_proportions': {str(k): float(v) for k, v in proportions.items()},
    'harm_counts_per_response': dict(harm_counts_per_response),
    'harm_counts_per_pair': dict(harm_counts_per_pair),
}
with open('data/alpaca7b_stats.json', 'w') as f:
    json.dump(stats, f, indent=2)
print('Wrote selections and stats.')


Alpaca-7B pair responses: 27393
Bucket counts:
  (False, False): 12689
  (False, True): 2099
  (True, False): 1645
  (True, True): 10960
Bucket proportions:
  (False, False): 0.463
  (False, True): 0.077
  (True, False): 0.060
  (True, True): 0.400
Top harm categories (per-response):
  Cybercrime: 8562
  Economic Crime: 7135
  Privacy Violation: 6296
  Mental Manipulation: 5311
  Violence: 4089
  Physical Harm: 4088
  Psychological Harm: 3727
  White-Collar Crime: 3533
  Insulting Behavior: 2962
  Drugs: 2636
  Endangering National Security: 2400
  Human Trafficking: 2051
  Disrupting Public Order: 1870
  Discriminatory Behavior: 1391
  Sexual Content: 873
  Copyright Issues: 776
  Endangering Public Health: 743
  Animal Abuse: 582
  Environmental Damage: 542
Top harm categories (per-pair):
  Cybercrime: 5390
  Economic Crime: 4297
  Privacy Violation: 4232
  Mental Manipulation: 3324
  Physical Harm: 2791
  Psychological Harm: 2716
  Violence: 2567
  White-Collar Crime: 2254
  Insulti

In [ ]:
# Compute the same statistics for the selected train and eval sets
from collections import Counter

def compute_selection_stats(samples):
    buckets = Counter((bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe'])) for ex in samples)
    total = sum(buckets.values()) or 1
    proportions = {k: v / total for k, v in buckets.items()}
    harm_resp = Counter()
    harm_pair = Counter()
    for ex in samples:
        s0, s1 = bool(ex['is_response_0_safe']), bool(ex['is_response_1_safe'])
        if s0 and s1:
            continue
        for fld in ['response_0_harm_category', 'response_1_harm_category']:
            cat = ex.get(fld, {}) or {}
            for name, flag in cat.items():
                if flag:
                    harm_resp[name] += 1
        pair_set = set()
        for fld in ['response_0_harm_category', 'response_1_harm_category']:
            cat = ex.get(fld, {}) or {}
            for name, flag in cat.items():
                if flag:
                    pair_set.add(name)
        for name in pair_set:
            harm_pair[name] += 1
    return buckets, proportions, harm_resp, harm_pair

train_b, train_p, train_hr, train_hp = compute_selection_stats(selected_train)
eval_b, eval_p, eval_hr, eval_hp = compute_selection_stats(selected_eval)

print('TRAIN bucket counts:')
for k, v in sorted(train_b.items()):
    print(f'  {k}: {v}')
print('TRAIN bucket proportions:')
for k, p in sorted(train_p.items()):
    print(f'  {k}: {p:.3f}')
print('Top TRAIN harms (per-response):')
for name, cnt in train_hr.most_common(20):
    print(f'  {name}: {cnt}')
print('Top TRAIN harms (per-pair):')
for name, cnt in train_hp.most_common(20):
    print(f'  {name}: {cnt}')

print('EVAL bucket counts:')
for k, v in sorted(eval_b.items()):
    print(f'  {k}: {v}')
print('EVAL bucket proportions:')
for k, p in sorted(eval_p.items()):
    print(f'  {k}: {p:.3f}')
print('Top EVAL harms (per-response):')
for name, cnt in eval_hr.most_common(20):
    print(f'  {name}: {cnt}')
print('Top EVAL harms (per-pair):')
for name, cnt in eval_hp.most_common(20):
    print(f'  {name}: {cnt}')

# Save selection stats
with open('data/custom_train_250_stats.json', 'w') as f:
    json.dump({
        'bucket_counts': {str(k): int(v) for k, v in train_b.items()},
        'bucket_proportions': {str(k): float(v) for k, v in train_p.items()},
        'harm_counts_per_response': dict(train_hr),
        'harm_counts_per_pair': dict(train_hp),
        'num_rows': len(selected_train),
    }, f, indent=2)
with open('data/custom_eval_500_stats.json', 'w') as f:
    json.dump({
        'bucket_counts': {str(k): int(v) for k, v in eval_b.items()},
        'bucket_proportions': {str(k): float(v) for k, v in eval_p.items()},
        'harm_counts_per_response': dict(eval_hr),
        'harm_counts_per_pair': dict(eval_hp),
        'num_rows': len(selected_eval),
    }, f, indent=2)
print('Saved selection stats JSON files.')


TRAIN bucket counts:
  (False, False): 116
  (False, True): 19
  (True, False): 15
  (True, True): 100
TRAIN bucket proportions:
  (False, False): 0.464
  (False, True): 0.076
  (True, False): 0.060
  (True, True): 0.400
Top TRAIN harms (per-response):
  Cybercrime: 90
  Privacy Violation: 68
  Mental Manipulation: 50
  Economic Crime: 50
  Physical Harm: 38
  Violence: 36
  Endangering National Security: 35
  Insulting Behavior: 31
  Psychological Harm: 30
  White-Collar Crime: 29
  Drugs: 23
  Disrupting Public Order: 16
  Discriminatory Behavior: 10
  Human Trafficking: 8
  Animal Abuse: 8
  Copyright Issues: 7
  Endangering Public Health: 7
  Environmental Damage: 3
  Sexual Content: 2
Top TRAIN harms (per-pair):
  Cybercrime: 56
  Privacy Violation: 43
  Economic Crime: 34
  Mental Manipulation: 31
  Physical Harm: 26
  Psychological Harm: 23
  Violence: 22
  Insulting Behavior: 21
  Endangering National Security: 18
  White-Collar Crime: 18
  Drugs: 12
  Disrupting Public Order: 